In [1]:
import os, sys; sys.path.append(os.path.dirname(os.getcwd()))
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import confusion_matrix, classification_report

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

import multimodal_vae
import torch_mvae_util as U
import util.RAVDESS_dataset_util as Rd
from train_mvae import build_model, train

In [2]:
from functools import partial
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from ray.tune import JupyterNotebookReporter

In [3]:
batch_size=512
num_workers=32
max_num_epochs=50
num_epochs=100
num_samples=50
data_dir='/home/studenti/ballerini/datasets/au-emo.csv'
checkpoint_dir='./tuning'

modes={'au':True, 'face':None, 'emotion':True}

config = {
    'batch_size':tune.choice([25, 50, 100, 150]),
    'latent_space_dim':tune.choice([25, 50, 100, 150]),
    'hidden_dim':tune.choice([64, 128, 256, 512]),
    'lr': tune.choice([1e-2, 1e-3, 1e-4, 1e-5]),
    'beta': tune.choice([1e-4, 1e-5, 1e-6, 1e-7]),
    'au_weight':tune.choice([1e3, 1e5, 1e6, 1e6]),
    'emotion_weight':tune.choice([1e3, 1e5, 1e6, 1e6]),
}

In [4]:
def load_data(data_dir='./data'):
    au_dataset = pd.read_csv(data_dir).to_numpy()
    au = au_dataset[:,:-1]
    emotions = au_dataset[:,-1].astype(int)-1
    au_dataset = [(x, y) for x, y in zip(au, emotions)]

    trainingset_len = int(len(au_dataset) * 0.8)
    testset_len = len(au_dataset) - trainingset_len

    trainset, testset = torch.utils.data.random_split(
        au_dataset, 
        [trainingset_len, testset_len],
        generator=torch.Generator().manual_seed(66)
    )
    
    return trainset, testset

In [5]:
def hyper_train(config, data_dir, num_epochs, use_cuda=True, checkpoint_dir=None):
    model: torch.nn.Module = build_model(
        cat_dim=8,      
        au_dim=18,     
        latent_space_dim=config['latent_space_dim'],     
        hidden_dim=config['hidden_dim'],               
        num_filters=None,                 
        modes=modes,              
        au_weight=config['au_weight'],           
        emotion_weight=config['emotion_weight'],              
        expert_type='poe',                  
        use_cuda=True).double()
        
    optimizer = torch.optim.Adam(
        params=model.parameters(), 
        lr=config['lr'], 
        betas=[0.95, 0.98])
    
    if checkpoint_dir:
        model_state, optimizer_state = torch.load(
            os.path.join(checkpoint_dir, "checkpoint"))
        model.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)
       
    trainset, testset = load_data(data_dir)
    
    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs])
    
    trainloader = DataLoader(
        trainset, 
        batch_size=config['batch_size'],
        shuffle=True, 
        num_workers=32)

    valloader = DataLoader(
        val_subset, 
        batch_size=config['batch_size'],
        shuffle=True, 
        num_workers=32)
    
    beta = config['beta']
    for epoch in range(num_epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        epoch_steps = 0
        
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            au, emotions = data
            
            if use_cuda:
                if au is not None:
                    au = au.cuda()
                if emotions is not None:
                    emotions = emotions.cuda()

            # zero the parameter gradients
            optimizer.zero_grad()
            
            (
                au_reconstruction,
                emotion_reconstruction,
                z_loc_expert,
                z_scale_expert,
                latent_sample
            ) = model(
                au=au,
                emotions=emotions
            )

            loss = model.loss_function(
                au=au,
                emotions=emotions,
                au_reconstruction=au_reconstruction,
                emotions_reconstruction=emotion_reconstruction,
                z_loc=z_loc_expert,
                z_scale=z_scale_expert,
                beta=beta,
                latent_sample=latent_sample
            )
            
            loss["total_loss"].backward()
            optimizer.step()

        y_true = []
        y_pred = []
        val_loss = 0
        val_steps = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                au, emotions = data
            
                if use_cuda:
                    if au is not None:
                        au = au.cuda()
                    if emotions is not None:
                        emotions = emotions.cuda()

                (
                    au_reconstruction,
                    emotion_reconstruction,
                    z_loc_expert,
                    z_scale_expert,
                    latent_sample
                ) = model(
                    au=au,
                    emotions=None
                )
                
                loss = model.loss_function(
                    au=au,
                    emotions=emotions,
                    au_reconstruction=au_reconstruction,
                    emotions_reconstruction=emotion_reconstruction,
                    z_loc=z_loc_expert,
                    z_scale=z_scale_expert,
                    beta=beta,
                    latent_sample=latent_sample
                )
                
                reconstructed_emotions = torch.argmax(emotion_reconstruction, 1)
                y_true += emotions.cpu()
                y_pred += reconstructed_emotions.cpu()
                val_loss += loss['total_loss'].cpu().numpy()
                val_steps += 1
                
        matrix = confusion_matrix(y_true, y_pred)
        accuracy = matrix.diagonal()/matrix.sum(axis=1)
        accuracy = sum(accuracy) / len(accuracy)
        
        with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((model.state_dict(), optimizer.state_dict()), path)

        tune.report(loss=(val_loss / val_steps), accuracy=accuracy)
    print("Finished Training")
    

In [6]:
#hyper_train(config, data_dir, num_epochs)

In [7]:
scheduler = ASHAScheduler(
    metric="accuracy",
    mode="max",
    max_t=max_num_epochs,
    grace_period=1,
    reduction_factor=2)    

reporter = JupyterNotebookReporter(
    True,
    metric_columns=["accuracy","loss","training_iteration"])

result = tune.run(
        partial(hyper_train, data_dir=data_dir, num_epochs=num_epochs, checkpoint_dir=checkpoint_dir),
        resources_per_trial={"cpu": 8, "gpu": 0.2},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter)

best_trial = result.get_best_trial("accuracy", "max", "last")

print("Best trial config: {}".format(best_trial.config))
print("Best trial final validation loss: {}".format(best_trial.last_result["loss"]))
print("Best trial final validation accuracy: {}".format(best_trial.last_result["accuracy"]))

Trial name,status,loc,au_weight,beta,emotion_weight,hidden_dim,latent_space_dim,lr,accuracy,loss,training_iteration
hyper_train_533b3_00000,TERMINATED,159.149.133.23:1390799,5787.53,1e-06,0.0175104,128,150,1e-05,0.2096,1790.63,32
hyper_train_533b3_00001,TERMINATED,159.149.133.23:1390830,7482.33,1e-06,833.83,128,100,1e-05,0.130333,10082.7,4
hyper_train_533b3_00002,TERMINATED,159.149.133.23:1390832,0.203147,1e-07,1822.08,64,50,0.01,0.142707,8004.04,2
hyper_train_533b3_00003,TERMINATED,159.149.133.23:1390834,44.8169,0.0001,10972.3,128,100,0.001,0.205498,28548.3,16
hyper_train_533b3_00004,TERMINATED,159.149.133.23:1390836,0.1085,1e-07,30787.1,512,100,1e-05,0.236723,61003,100
hyper_train_533b3_00005,TERMINATED,159.149.133.23:1432829,0.026724,0.0001,1.11097,64,100,0.0001,0.277925,2.21721,32
hyper_train_533b3_00006,TERMINATED,159.149.133.23:1451066,0.105847,0.0001,4.01938,64,25,0.0001,0.30031,8.15159,100
hyper_train_533b3_00007,TERMINATED,159.149.133.23:1641600,101.749,0.0001,0.0319115,256,100,0.0001,0.131105,73.5261,1
hyper_train_533b3_00008,TERMINATED,159.149.133.23:1682384,251207,1e-07,0.0496867,64,100,0.001,0.318188,3483.67,100
hyper_train_533b3_00009,TERMINATED,159.149.133.23:1876954,0.284532,1e-07,67423.8,512,25,0.001,0.238262,195966,8


2022-07-04 14:15:00,188	INFO tune.py:747 -- Total run time: 1117.02 seconds (1116.85 seconds for the tuning loop).


Best trial config: {'latent_space_dim': 25, 'hidden_dim': 64, 'lr': 0.001, 'beta': 0.0001, 'au_weight': 83710.36378474736, 'emotion_weight': 0.2905421211580692}
Best trial final validation loss: 1159.7698166428454
Best trial final validation accuracy: 0.5992605533154622
